In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
.master("local[4]") \
.appName("pyspark_Df_to_disk") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

In [4]:
sc = spark.sparkContext

In [5]:
df = spark.read \
.option("header","True") \
.option("inferSchema","True")\
.option("sep",",") \
.csv("/home/taha/Downloads/simple_dirty_data.csv")

In [6]:
df.show(2)

+------+------+---+--------+------+-------+-----------+--------+
|sirano|  isim|yas|cinsiyet|meslek|  sehir|aylik_gelir|mal_mulk|
+------+------+---+--------+------+-------+-----------+--------+
|     1| Cemal| 35|       E|  Isci| Ankara|     3500.0|   araba|
|     2|ceyda | 42|       K| Memur|Kayseri|     4200.0| arabaev|
+------+------+---+--------+------+-------+-----------+--------+
only showing top 2 rows



# Veri Temizligi

In [7]:
from pyspark.sql import functions as F

In [11]:
df2 = df \
.withColumn("isim",F.trim(F.initcap(df.isim))) \
.withColumn("cinsiyet",F.when(df.cinsiyet.isNull(),"U").otherwise(df.cinsiyet)) \
.withColumn("sehir",F.when(df.sehir.isNull(),"BİLİNMİYOR").otherwise(F.trim(F.upper(df.sehir))))

df2.show(2)

+------+-----+---+--------+------+-------+-----------+--------+
|sirano| isim|yas|cinsiyet|meslek|  sehir|aylik_gelir|mal_mulk|
+------+-----+---+--------+------+-------+-----------+--------+
|     1|Cemal| 35|       E|  Isci| ANKARA|     3500.0|   araba|
|     2|Ceyda| 42|       K| Memur|KAYSERI|     4200.0| arabaev|
+------+-----+---+--------+------+-------+-----------+--------+
only showing top 2 rows



# Temizlenmis dataframe i diske yaz

In [18]:
# coalesce(1) parca parca yazmasını sagliypr
df2 \
.coalesce(1) \
.write \
.mode("overwrite") \
.option("header","True") \
.option("sep",",") \
.csv("/home/taha/Downloads/temiz_yazdirma")

In [19]:
df = spark.read \
.option("header","True") \
.option("inferSchema","True")\
.option("sep",",") \
.csv("/home/taha/Downloads/temiz_yazdirma")

df.show(2)

+------+-----+---+--------+------+-------+-----------+--------+
|sirano| isim|yas|cinsiyet|meslek|  sehir|aylik_gelir|mal_mulk|
+------+-----+---+--------+------+-------+-----------+--------+
|     1|Cemal| 35|       E|  Isci| ANKARA|     3500.0|   araba|
|     2|Ceyda| 42|       K| Memur|KAYSERI|     4200.0| arabaev|
+------+-----+---+--------+------+-------+-----------+--------+
only showing top 2 rows

